In [1]:
import pandas as pd
from collections import Counter
import os

In [2]:
df = pd.read_csv('merged_Friday.csv')

In [3]:
# Separate normal and non-normal instances
normal_traffic = df[df[' Label'] == 'BENIGN']  # Assuming 'benign' corresponds to normal traffic
non_normal_traffic = df[df[' Label'] != 'BENIGN']

# Randomly select 20000 instances from normal traffic
downsampled_normal_traffic = normal_traffic.sample(n=10000, random_state=42)

# Combine downsampled normal traffic with non-normal traffic
downsampled_df = pd.concat([downsampled_normal_traffic, non_normal_traffic])

# Shuffle the combined DataFrame
downsampled_df = downsampled_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:
# Divide each value in the 'Fwd Packet Length Mean' column by the corresponding value in the 'Total Fwd Packets' column
downsampled_df['Fwd Packets Normalized'] = (downsampled_df[' Fwd Packet Length Mean'] / downsampled_df[' Total Fwd Packets']).round(3)

downsampled_df['Bwd Packets Normalized'] = (downsampled_df[' Bwd Packet Length Mean'] / downsampled_df[' Total Backward Packets']).round(3)

In [5]:
# Find the maximum value in a column
maximum_value = downsampled_df['Fwd Packets Normalized'].max()

# Find the minimum value in a column
minimum_value = downsampled_df['Fwd Packets Normalized'].min()

# Print the maximum and minimum values
print("Maximum Value:", maximum_value)
print("Minimum Value:", minimum_value)

Maximum Value: 1448.0
Minimum Value: 0.0


In [6]:
# Find the maximum value in a column
maximum_value = downsampled_df['Bwd Packets Normalized'].max()

# Find the minimum value in a column
minimum_value = downsampled_df['Bwd Packets Normalized'].min()

# Print the maximum and minimum values
print("Maximum Value:", maximum_value)
print("Minimum Value:", minimum_value)

Maximum Value: 2900.25
Minimum Value: 0.0


In [7]:
import numpy as np
# Define ranges for categorization
category_ranges = {
    'fwd_low': (0, 1000.00),
    'fwd_medium': (1001.00, 2000.00),
    'fwd_high': (2001.00, 3000.00),
    'Bwd_very_high': (3001.00, np.inf) 
}

# Function to categorize values based on ranges
def categorize_value(value, ranges):
    if pd.isnull(value):
        return 'fwd_low'  # Treat NaN values as 'low'
    for category, (lower, upper) in ranges.items():
        if lower <= value <= upper:
            return category
    return 'unknown'

# Apply categorization to 'Fwd Packets Normalized' column
downsampled_df['Fwd Packets Category'] = downsampled_df['Fwd Packets Normalized'].apply(lambda x: categorize_value(x, category_ranges))

In [8]:
import numpy as np
# Define ranges for categorization
category_ranges = {
    'Bwd_low': (0, 1000.00),
    'Bwd_medium': (1001.00, 2000.00),
    'Bwd_high': (2001.00, 3000.00),
    'Bwd_very_high' : (3001.00, np.inf)
}

# Function to categorize values based on ranges
def categorize_value(value, ranges):
    if pd.isnull(value):
        return 'Bwd_low'  # Treat NaN values as 'low'
    for category, (lower, upper) in ranges.items():
        if lower <= value <= upper:
            return category
    return 'unknown'

# Apply categorization to 'Bwd Packets Normalized' column
downsampled_df['Bwd Packets Category'] = downsampled_df['Bwd Packets Normalized'].apply(lambda x: categorize_value(x, category_ranges))

In [9]:
import numpy as np
# Define ranges for categorization
ranges = {
    '0_750': (0.0, 750.0),
    '751_1500': (750.0, 1500.0),
    '1501_15000': (1500.0, 15000.0),
    '15001_120000': (15000.0, 120000.0),
    '120001_300000': (120000.0, 300000.0),
    '300001_6000000': (300000.0, 6000000.0),
    '6000001_65999946': (6000000.0, 65999946.0),
    '66000000_119999902': (65999946.0, 119999902.0),
    '119999902_infinity': (119999902.0, np.inf)
}

# Function to categorize values based on ranges
def categorize_value(value, ranges):
    if pd.isnull(value):
        return 'unknown'  # Treat NaN values as 'unknown'
    for category, (lower, upper) in ranges.items():
        if lower <= value <= upper:
            return category
    return 'unknown'

# Apply categorization to 'Flow Duration' column
downsampled_df['Flow Duration Category'] = downsampled_df[' Flow Duration'].apply(lambda x: categorize_value(x, ranges))



In [10]:
essential_columns = [' Source IP', ' Destination IP',' Destination Port', ' Protocol', 'Fwd Packets Category', 'Bwd Packets Category', 'Flow Duration Category']

In [11]:
# Drop non-essential columns
filtered_df = downsampled_df[essential_columns]
print("\nFiltered dataset Summery")
# Function to convert IP addresses to first subnet
def convert_to_first_subnet(ip_address):
    octets = ip_address.split('.')
    octets[2] = '0'
    octets[3] = '0'
    return '.'.join(octets)


# Strip leading and trailing spaces from 'Source IP' and 'Destination IP' columns
filtered_df[' Source IP'] = filtered_df[' Source IP'].astype(str).str.strip()
filtered_df[' Destination IP'] = filtered_df[' Destination IP'].astype(str).str.strip()

# Apply the function to 'Source IP' and 'Destination IP' columns
filtered_df[' Source IP'] = filtered_df[' Source IP'].apply(convert_to_first_subnet)
filtered_df[' Destination IP'] = filtered_df[' Destination IP'].apply(convert_to_first_subnet)

# Check the first few rows of the DataFrame to verify the changes
print("\nFiltered dataset after clearing subnets for first 2 octets")
print(filtered_df.head())


Filtered dataset Summery


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11880\4025526751.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[' Source IP'] = filtered_df[' Source IP'].astype(str).str.strip()
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11880\4025526751.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[' Destination IP'] = filtered_df[' Destination IP'].astype(str).str.strip()
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11880\4025526751.py:17: SettingWithCopyWarning: 
A value is trying to be set 


Filtered dataset after clearing subnets for first 2 octets
    Source IP  Destination IP   Destination Port   Protocol  \
0  172.16.0.0     192.168.0.0                 80          6   
1  172.16.0.0     192.168.0.0               6788          6   
2  172.16.0.0     192.168.0.0                 80          6   
3  172.16.0.0     192.168.0.0                800          6   
4  172.16.0.0     192.168.0.0               5102          6   

  Fwd Packets Category Bwd Packets Category Flow Duration Category  
0              fwd_low              Bwd_low     66000000_119999902  
1              fwd_low              Bwd_low                  0_750  
2              fwd_low              Bwd_low       6000001_65999946  
3              fwd_low              Bwd_low                  0_750  
4              fwd_low              Bwd_low                  0_750  


In [12]:
# Convert first five columns to strings and concatenate with hyphens
filtered_df['FlowID'] = filtered_df.iloc[:, :2].astype(str).apply('-'.join, axis=1)

# Drop the original columns
filtered_df = filtered_df.drop(columns=[' Source IP', ' Destination IP']) # , ' Source Port'

# Display the DataFrame
filtered_df.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11880\571974056.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['FlowID'] = filtered_df.iloc[:, :2].astype(str).apply('-'.join, axis=1)


,Destination Port,Protocol,Fwd Packets Category,Bwd Packets Category,Flow Duration Category,FlowID
0,80,6,fwd_low,Bwd_low,66000000_119999902,172.16.0.0-192.168.0.0
1,6788,6,fwd_low,Bwd_low,0_750,172.16.0.0-192.168.0.0
2,80,6,fwd_low,Bwd_low,6000001_65999946,172.16.0.0-192.168.0.0
3,800,6,fwd_low,Bwd_low,0_750,172.16.0.0-192.168.0.0
4,5102,6,fwd_low,Bwd_low,0_750,172.16.0.0-192.168.0.0


In [13]:
with_ID = filtered_df.copy()
with_ID.insert(0, 'alertID', range(len(with_ID)))
print("\nAdding a ID number for the records")
print(with_ID.head())

dataset_with_ID = with_ID.values.tolist() 


Adding a ID number for the records
   alertID   Destination Port   Protocol Fwd Packets Category  \
0        0                 80          6              fwd_low   
1        1               6788          6              fwd_low   
2        2                 80          6              fwd_low   
3        3                800          6              fwd_low   
4        4               5102          6              fwd_low   

  Bwd Packets Category Flow Duration Category                  FlowID  
0              Bwd_low     66000000_119999902  172.16.0.0-192.168.0.0  
1              Bwd_low                  0_750  172.16.0.0-192.168.0.0  
2              Bwd_low       6000001_65999946  172.16.0.0-192.168.0.0  
3              Bwd_low                  0_750  172.16.0.0-192.168.0.0  
4              Bwd_low                  0_750  172.16.0.0-192.168.0.0  


In [14]:
print("\nconverting the dataset in to a list")
dataset = [alert[1:] for alert in dataset_with_ID]


converting the dataset in to a list


In [15]:
# Convert each sublist to a tuple for hashability
data_tuples = [tuple(sublist) for sublist in dataset]

# Count occurrences of each unique record
record_counts = Counter(data_tuples)

print("\nnumber occurrences of each unique record")
print(f"{len(record_counts) }")


number occurrences of each unique record
5670


In [16]:
import pandas as pd

# Assuming 'filtered_df' is your DataFrame with categorical values

# Initialize dictionaries to store both forward and reverse mappings
forward_mapping = {}
reverse_mapping = {}

# Initialize the global counter to keep track of numerical values
global_counter = 0

# Iterate over each column in the DataFrame
for column in filtered_df.columns:
    # Initialize a local counter for each column
    local_counter = 0
    
    # Initialize dictionaries for forward and reverse mappings for the current column
    forward_mapping[column] = {}
    reverse_mapping[column] = {}
    
    # Iterate over each unique value in the current column
    for value in filtered_df[column].unique():
        # Map each unique value to a numerical value based on the global counter
        forward_mapping[column][value] = global_counter + local_counter
        reverse_mapping[column][global_counter + local_counter] = value
        
        # Increment the local counter
        local_counter += 1
        
    # Update the global counter to continue numbering from the last value of the previous dictionary
    global_counter += local_counter

# Create a new DataFrame with mapped values
new_df = pd.DataFrame()

# Iterate over each column in the original DataFrame and fill in values in the new DataFrame
for column in filtered_df.columns:
    new_df[column] = filtered_df[column].map(forward_mapping[column])

# Display the DataFrame with numerical values
print(new_df)

# Display forward and reverse mappings for each column
for column in new_df.columns:
    print(f"For {column}:")
    print("Forward Mapping:", forward_mapping[column])
    print("Reverse Mapping:", reverse_mapping[column])
    print()


         Destination Port   Protocol  Fwd Packets Category  \
0                       0       3808                  3811   
1                       1       3808                  3811   
2                       0       3808                  3811   
3                       2       3808                  3811   
4                       3       3808                  3811   
...                   ...        ...                   ...   
298918                  0       3808                  3811   
298919               1062       3808                  3811   
298920                  0       3808                  3811   
298921                714       3808                  3811   
298922                  0       3808                  3811   

        Bwd Packets Category  Flow Duration Category  FlowID  
0                       3813                    3816    3824  
1                       3813                    3817    3824  
2                       3813                    3818    3824  
3  

In [17]:
# Calculate the count of unique values in the 'type' field
type_counts = downsampled_df[' Label'].value_counts()

print("\nDisplay the count of unique values in Label")
print(type_counts)


Display the count of unique values in Label
 Label
PortScan    158930
DDoS        128027
BENIGN       10000
Bot           1966
Name: count, dtype: int64


In [18]:
combined_dict = {k: v for inner_dict in reverse_mapping.values() for k, v in inner_dict.items()}

item_dataset = [tuple(x) for x in new_df.to_records(index=False)]

# Add ID field to the start of each tuple
item_dataset_withID = [(i,) + record for i, record in enumerate(item_dataset)]

new_df.to_csv('data.txt', index=False, sep=' ', header=False)

In [23]:
# Run the algorithm

os.system("java -jar spmf.jar run FPMax data.txt output.txt 1%")

itemset_records_object = []
itemset_records_numbers = []

# Read the output file line by line
outFile = open("output.txt", 'r', encoding='utf-8')
for string in outFile:
    itemset = []
    parts = string.split('#SUP:')
    numbers = list(map(int, parts[0].split()))
    support_count = int(parts[1].strip())

    itemset_using_numbers = [numbers, support_count]
    itemset_records_numbers.append(itemset_using_numbers)

    # Translate numerical values to attribute names using reverse mapping
    attribute_names = [str(combined_dict[num]) for num in numbers]
    itemset = [attribute_names, support_count]
    # Output the result
    #print(f"Pattern: {' '.join(attribute_names)}, Support Count: {str(support_count)}")
    itemset_records_object.append(itemset)

outFile.close()

def return_unique_labels(alertID_List):
    # Filter DataFrame based on selected IDs
    selected_records = downsampled_df.iloc[alertID_List]
    # Count unique values in a certain field (e.g., Field1) in the selected records
    unique_value_counts = selected_records[' Label'].value_counts().to_dict()
    return(unique_value_counts)

patterns_ID_list = []
pattern_label_list = []

for index,record in enumerate(itemset_records_numbers):
    itemset = record[0]
    containing_alerts = []
    for alert in item_dataset_withID:
        alert_items = set(alert[1:])  # Exclude the ID field
        if set(itemset).issubset(alert_items):
            containing_alerts.append(alert[0])  # Append the ID

    pattern_label_record = list(return_unique_labels(containing_alerts).items())
    patterns_ID_list.append((itemset_records_object[index][0],containing_alerts))
    pattern_label_list.append((itemset_records_object[index][0],pattern_label_record))
    print(f"Pattern {index}: {itemset_records_object[index][0]}, \n{pattern_label_record}\n===============================================================================================================")


Pattern 0: ['fwd_low', 'Bwd_low', '17', '53'], 
[('BENIGN', 4342)]
Pattern 1: ['fwd_low', 'Bwd_low', '6', '172.16.0.0-192.168.0.0', '80', '120001_300000'], 
[('DDoS', 6563), ('PortScan', 3)]
Pattern 2: ['fwd_low', 'Bwd_low', '6', '172.16.0.0-192.168.0.0', '80', '15001_120000'], 
[('DDoS', 16542), ('PortScan', 34)]
Pattern 3: ['fwd_low', 'Bwd_low', '6', '172.16.0.0-192.168.0.0', '80', '66000000_119999902'], 
[('DDoS', 21370)]
Pattern 4: ['fwd_low', 'Bwd_low', '6', '172.16.0.0-192.168.0.0', '80', '6000001_65999946'], 
[('DDoS', 25720), ('PortScan', 29)]
Pattern 5: ['fwd_low', 'Bwd_low', '6', '172.16.0.0-192.168.0.0', '80', '300001_6000000'], 
[('DDoS', 56867), ('PortScan', 28), ('BENIGN', 1)]
Pattern 6: ['fwd_low', 'Bwd_low', '6', '172.16.0.0-192.168.0.0', '0_750'], 
[('PortScan', 158106), ('DDoS', 75)]


In [24]:
# Function to get the original field and value using the reverse mapping dictionaries
def get_field_and_value(item):
    for column, mapping in reverse_mapping.items():
        if int(item) in mapping:
            return str(column), mapping[int(item)]
    return None, None

In [25]:
# Initialize a list to store the patterns with original field and value

pattern_record = pd.DataFrame(columns=['Support Count', 'Label'])
patterns_with_fields = []

for index, record in enumerate(itemset_records_numbers):
    # Split the line into items and support count
    items = record[0]
    support_count = int(record[1])

    # Initialize a list to store the original field and value of each item
    record_with_field = []

    # Map each item in the pattern to its original field and value
    for item in items:
        # Get the field and value using the reverse mapping dictionaries
        field, value = get_field_and_value(item)
        record_with_field.append({"field": field, "value": value})

        # Check if the field already exists in the DataFrame
        if field not in pattern_record.columns:
            # If not, add a new column with the field name and fill with NaN
            pattern_record[field] = pd.NA
        
        # Add the value to the corresponding field
        pattern_record.at[index, field] = value
    
        # Add the support_count to the pattern_data dictionary
    pattern_record.at[index, 'Support Count'] = support_count

    if len(pattern_label_list[index][1]) == 1:
        pattern_record.at[index, 'Label'] = pattern_label_list[index][1][0][0]
    else:
        pattern_record.at[index, 'Label'] = 'Mixed Labels'
    # Store the pattern with original field and value along with the support count
    
    patterns_with_fields.append({"pattern": record_with_field, "support_count": support_count})

pattern_record = pattern_record.fillna('NaN').infer_objects(copy=False)

# Print the patterns with original field and value
for pattern_info in patterns_with_fields:
    print(pattern_info)
    print()


{'pattern': [{'field': 'Fwd Packets Category', 'value': 'fwd_low'}, {'field': 'Bwd Packets Category', 'value': 'Bwd_low'}, {'field': ' Protocol', 'value': 17}, {'field': ' Destination Port', 'value': 53}], 'support_count': 4342}

{'pattern': [{'field': 'Fwd Packets Category', 'value': 'fwd_low'}, {'field': 'Bwd Packets Category', 'value': 'Bwd_low'}, {'field': ' Protocol', 'value': 6}, {'field': 'FlowID', 'value': '172.16.0.0-192.168.0.0'}, {'field': ' Destination Port', 'value': 80}, {'field': 'Flow Duration Category', 'value': '120001_300000'}], 'support_count': 6566}

{'pattern': [{'field': 'Fwd Packets Category', 'value': 'fwd_low'}, {'field': 'Bwd Packets Category', 'value': 'Bwd_low'}, {'field': ' Protocol', 'value': 6}, {'field': 'FlowID', 'value': '172.16.0.0-192.168.0.0'}, {'field': ' Destination Port', 'value': 80}, {'field': 'Flow Duration Category', 'value': '15001_120000'}], 'support_count': 16576}

{'pattern': [{'field': 'Fwd Packets Category', 'value': 'fwd_low'}, {'fiel